<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-12 15:30:56
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.39 C
-------------------
Today PnL: -1.71 L (-1.22%)
Current PnL: -30.38 L (-19.38%)
CY Booked + Current PnL: -15.64 L (-9.98%)
-------------------
Total profit:  1.56 L
Total loss:  -31.94 L
-------------------
Total Booked + Current PnL: 11.34 L (8.73%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.6%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 95.76 L (68.84%)
Deployed:  1.30 C
Current:  1.39 C
CAGR/XIRR %: 3.57%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,1.97,2.97,18.06,21.57,18221.0,2910.0,100890.0,397.04,-11.02,61.0,H-SC,3.54,168.0,0.16,0.73,47.23,XR,ATH,FINANCE
18,COALINDIA,-0.96,3.09,15.66,19.23,24289.0,4644.0,155104.0,484.83,21.35,47.0,L-LC,12.08,182.0,0.19,1.12,25.02,XY25,ATH,MINING
77,TTKPRESTIG,-1.42,-24.61,32.76,0.09,24891.0,-24797.0,75980.0,770.00,76.31,47.0,M-SC,8.54,253.0,-1.00,0.55,4.57,OX40N,NTT,DURABLES
5,ANGELONE,-0.91,12.85,10.06,24.20,34654.0,39225.0,344475.0,3033.00,60.95,60.0,X-SC,3.59,99.0,1.13,2.48,44.27,X40N,NTT,FINANCE
51,MEDANTA,0.24,-4.06,29.33,24.08,35722.0,-5156.0,121794.0,1486.00,-11.91,54.0,X-SC,11.62,91.0,-0.14,0.88,11.55,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,3.06,-21.25,104.15,60.77,149911.0,-38837.0,143938.0,521.00,65.84,61.0,H-SC,2.19,158.0,-0.26,1.03,30.50,XR,NTT,JEWELLERY
85,ZYDUSLIFE,2.36,-3.04,39.88,35.63,81040.0,-6364.0,203210.0,1286.17,-16.74,56.0,H-MC,5.98,119.0,-0.08,1.46,13.13,AR,ATH,PHARMA
50,MASFIN,1.97,2.97,18.06,21.57,18221.0,2910.0,100890.0,397.04,-11.02,61.0,H-SC,3.54,168.0,0.16,0.73,47.23,XR,ATH,FINANCE
82,VOLTAS,1.83,20.01,20.99,45.20,48298.0,38358.0,230100.0,1856.00,8.32,68.0,X-MC,3.09,80.0,0.79,1.65,28.16,XY25,NTT,AC
7,ATULAUTO,1.57,-12.35,66.08,45.57,114178.0,-24344.0,172788.0,844.00,3748.65,70.0,M-SC,6.36,245.0,-0.21,1.24,30.76,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BSOFT,-7.32,-21.86,92.97,50.79,102179.0,-30748.0,109905.0,831.70,0.12,50.0,H-SC,7.83,171.0,-0.30,0.79,27.97,XR,ATH,IT
64,SHALBY,-6.58,-35.82,94.93,25.10,110357.0,-64892.0,116251.0,327.00,754.86,40.0,M-SC,15.99,248.0,-0.59,0.84,1.82,XY24,NTT,HEALTHCARE
42,INFY,-5.97,-5.78,42.48,34.24,125237.0,-18075.0,294813.0,1972.00,-26.78,21.0,X-LC,5.06,6.0,-0.14,2.12,2.03,X40,NTT,IT
73,TCS,-5.77,-24.61,57.25,18.55,163253.0,-93079.0,285158.0,4311.58,-34.95,27.0,X-LC,1.98,3.0,-0.57,2.05,0.00,X40,ATH,IT
72,TATAELXSI,-5.39,-28.41,84.42,32.03,79677.0,-37453.0,94382.0,9161.00,-22.59,35.0,H-SC,6.15,157.0,-0.47,0.68,5.72,OX40N,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.02,3.70,100.49,107.91,153856.0,5462.0,153106.0,1641.55,-15.69,67.0,M-SC,10.19,220.0,0.04,1.10,15.19,OX40N,ATH,CABLES
66,SIS,-1.52,-21.75,54.98,21.26,47766.0,-24154.0,86878.0,528.00,2052.85,58.0,H-SC,4.07,163.0,-0.51,0.62,19.77,OX40N,NTT,MISC
77,TTKPRESTIG,-1.42,-24.61,32.76,0.09,24891.0,-24797.0,75980.0,770.00,76.31,47.0,M-SC,8.54,253.0,-1.00,0.55,4.57,OX40N,NTT,DURABLES
72,TATAELXSI,-5.39,-28.41,84.42,32.03,79677.0,-37453.0,94382.0,9161.00,-22.59,35.0,H-SC,6.15,157.0,-0.47,0.68,5.72,OX40N,NTT,IT
70,SYMPHONY,-1.71,-28.97,40.73,-0.03,49515.0,-49573.0,121568.0,1306.00,-43.04,56.0,M-SC,10.62,206.0,-1.00,0.87,12.34,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1.83,20.01,20.99,45.2,48298.0,38358.0,230100.0,1856.0,8.32,68.0,X-MC,3.09,80.0,0.79,1.65,28.16,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.96,3.09,15.66,19.23,24289.0,4644.0,155104.0,484.83,21.35,47.0,L-LC,12.08,182.0,0.19,1.12,25.02,XY25,ATH,MINING
50,MASFIN,1.97,2.97,18.06,21.57,18221.0,2910.0,100890.0,397.04,-11.02,61.0,H-SC,3.54,168.0,0.16,0.73,47.23,XR,ATH,FINANCE
26,FINCABLES,0.02,3.70,100.49,107.91,153856.0,5462.0,153106.0,1641.55,-15.69,67.0,M-SC,10.19,220.0,0.04,1.10,15.19,OX40N,ATH,CABLES
1,ABB,-0.52,6.32,36.91,45.57,102669.0,16541.0,278160.0,7934.00,-31.69,69.0,H-MC,3.20,121.0,0.16,2.00,23.52,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.52,6.32,36.91,45.57,102669.0,16541.0,278160.0,7934.00,-31.69,69.0,H-MC,3.20,121.0,0.16,2.00,23.52,AR,NTT,ELECTRICAL
50,MASFIN,1.97,2.97,18.06,21.57,18221.0,2910.0,100890.0,397.04,-11.02,61.0,H-SC,3.54,168.0,0.16,0.73,47.23,XR,ATH,FINANCE
26,FINCABLES,0.02,3.70,100.49,107.91,153856.0,5462.0,153106.0,1641.55,-15.69,67.0,M-SC,10.19,220.0,0.04,1.10,15.19,OX40N,ATH,CABLES
39,INDIAMART,-1.91,-5.45,118.64,106.72,138347.0,-6725.0,116611.0,4810.62,-53.11,47.0,H-SC,7.55,138.0,-0.05,0.84,16.65,AR,ATH,MISC
85,ZYDUSLIFE,2.36,-3.04,39.88,35.63,81040.0,-6364.0,203210.0,1286.17,-16.74,56.0,H-MC,5.98,119.0,-0.08,1.46,13.13,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-5.97,-5.78,42.48,34.24,125237.0,-18075.0,294813.0,1972.00,-26.78,21.0,X-LC,5.06,6.0,-0.14,2.12,2.03,X40,NTT,IT
33,HCLTECH,-5.20,-4.54,29.73,23.83,68656.0,-10993.0,230931.0,1908.19,-0.38,23.0,X-LC,5.93,13.0,-0.16,1.66,10.80,X40,ATH,IT
73,TCS,-5.77,-24.61,57.25,18.55,163253.0,-93079.0,285158.0,4311.58,-34.95,27.0,X-LC,1.98,3.0,-0.57,2.05,0.00,X40,ATH,IT
13,BERGEPAINT,0.34,-18.04,47.81,21.14,89080.0,-41019.0,186320.0,680.00,-23.89,28.0,X-MC,10.47,75.0,-0.46,1.34,0.34,XY24,NTT,PAINTS
8,AWL,-1.47,-33.07,134.87,57.19,294381.0,-107857.0,218270.0,485.00,-68.35,30.0,X-MC,17.09,58.0,-0.37,1.57,0.00,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,-5.77,-24.61,57.25,18.55,163253.0,-93079.0,285158.0,4311.58,-34.95,27.0,X-LC,1.98,3.0,-0.57,2.05,0.00,X40,ATH,IT
81,VBL,-0.42,-8.09,47.61,35.67,138207.0,-25552.0,290290.0,671.64,-19.05,41.0,X-LC,2.01,5.0,-0.18,2.09,4.64,X40N,ATH,FMCG
21,DABUR,-0.50,1.62,41.41,43.70,104386.0,4011.0,252079.0,735.00,-6.68,57.0,X-MC,2.02,73.0,0.04,1.81,16.91,XY24,BTT,FMCG
34,HINDUNILVR,-2.27,-6.37,21.40,13.67,51510.0,-16363.0,240700.0,2922.00,-13.44,53.0,X-LC,2.57,9.0,-0.32,1.73,13.63,XY25,NTT,FMCG
65,SIEMENS,-0.02,-15.34,48.25,25.51,76018.0,-28545.0,157550.0,4671.50,-3.29,55.0,H-LC,2.88,50.0,-0.38,1.13,16.10,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,-1.47,-33.07,134.87,57.19,294381.0,-107857.0,218270.0,485.00,-68.35,30.0,X-MC,17.09,58.0,-0.37,1.57,0.00,XY24,NTT,FMCG
73,TCS,-5.77,-24.61,57.25,18.55,163253.0,-93079.0,285158.0,4311.58,-34.95,27.0,X-LC,1.98,3.0,-0.57,2.05,0.00,X40,ATH,IT
2,ABBOTINDIA,-1.02,-10.63,32.11,18.07,51325.0,-19015.0,159840.0,35195.00,-23.50,37.0,X-MC,9.95,64.0,-0.37,1.15,0.02,X40,ATH,PHARMA
13,BERGEPAINT,0.34,-18.04,47.81,21.14,89080.0,-41019.0,186320.0,680.00,-23.89,28.0,X-MC,10.47,75.0,-0.46,1.34,0.34,XY24,NTT,PAINTS
53,PGHH,-1.34,-12.25,50.44,32.01,94504.0,-26160.0,187360.0,17616.87,-38.28,38.0,X-MC,9.06,60.0,-0.28,1.35,1.27,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-2.26,-30.60,105.83,42.84,47744.0,-19892.0,45114.0,424.00,-54.82,50.0,X-SC,19.98,83.0,-0.42,0.32,8.52,XY24,NTT,MISC
58,RELAXO,-1.75,-52.73,222.19,52.31,152467.0,-76540.0,68620.0,1176.00,-49.67,39.0,X-SC,13.46,92.0,-0.50,0.49,2.04,X40N,NTT,FOOTWEAR
12,BATAINDIA,-2.76,-43.23,138.17,35.20,100925.0,-55626.0,73044.0,2096.00,-4.26,46.0,X-SC,20.97,93.0,-0.55,0.53,3.80,X40,NTT,FOOTWEAR
51,MEDANTA,0.24,-4.06,29.33,24.08,35722.0,-5156.0,121794.0,1486.00,-11.91,54.0,X-SC,11.62,91.0,-0.14,0.88,11.55,XY24,NTT,HEALTHCARE
35,HONAUT,-1.21,-21.83,83.34,43.32,106108.0,-35552.0,127320.0,58357.33,-32.87,41.0,X-SC,10.02,90.0,-0.34,0.92,3.38,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.24,-22.18,42.36,10.78,79217.0,-53310.0,187008.0,452.00,-53.82,39.0,X-LC,18.78,1.0,-0.67,1.34,3.00,X40,NTT,FMCG
75,TMPV,-0.38,-21.45,56.56,22.98,100363.0,-48453.0,177445.0,600.00,-18.19,67.0,X-LC,3.46,2.0,-0.48,1.28,13.44,XY24,NTT,AUTO
73,TCS,-5.77,-24.61,57.25,18.55,163253.0,-93079.0,285158.0,4311.58,-34.95,27.0,X-LC,1.98,3.0,-0.57,2.05,0.00,X40,ATH,IT
81,VBL,-0.42,-8.09,47.61,35.67,138207.0,-25552.0,290290.0,671.64,-19.05,41.0,X-LC,2.01,5.0,-0.18,2.09,4.64,X40N,ATH,FMCG
42,INFY,-5.97,-5.78,42.48,34.24,125237.0,-18075.0,294813.0,1972.00,-26.78,21.0,X-LC,5.06,6.0,-0.14,2.12,2.03,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.25,-31.88,95.08,32.89,50006.0,-24612.0,52594.0,1800.00,-749.28,57.0,L-MC,11.44,259.0,-0.49,0.38,44.84,XR,NTT,BANKS
67,SONACOMS,-0.94,-7.80,50.86,39.09,47435.0,-7893.0,93266.0,804.02,-27.83,66.0,M-SC,3.72,227.0,-0.17,0.67,31.94,AR,ATH,AUTO
50,MASFIN,1.97,2.97,18.06,21.57,18221.0,2910.0,100890.0,397.04,-11.02,61.0,H-SC,3.54,168.0,0.16,0.73,47.23,XR,ATH,FINANCE
14,BSOFT,-7.32,-21.86,92.97,50.79,102179.0,-30748.0,109905.0,831.70,0.12,50.0,H-SC,7.83,171.0,-0.30,0.79,27.97,XR,ATH,IT
79,VAIBHAVGBL,3.06,-21.25,104.15,60.77,149911.0,-38837.0,143938.0,521.00,65.84,61.0,H-SC,2.19,158.0,-0.26,1.03,30.50,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,1.03,-18.22,135.29,92.42,308191.0,-50759.0,227800.0,400.00,60.20,74.0,H-SC,8.32,169.0,-0.16,1.64,31.65,XY24,NTT,BANKS
5,ANGELONE,-0.91,12.85,10.06,24.20,34654.0,39225.0,344475.0,3033.00,60.95,60.0,X-SC,3.59,99.0,1.13,2.48,44.27,X40N,NTT,FINANCE
67,SONACOMS,-0.94,-7.80,50.86,39.09,47435.0,-7893.0,93266.0,804.02,-27.83,66.0,M-SC,3.72,227.0,-0.17,0.67,31.94,AR,ATH,AUTO
7,ATULAUTO,1.57,-12.35,66.08,45.57,114178.0,-24344.0,172788.0,844.00,3748.65,70.0,M-SC,6.36,245.0,-0.21,1.24,30.76,XY24,NTT,AUTO
1,ABB,-0.52,6.32,36.91,45.57,102669.0,16541.0,278160.0,7934.00,-31.69,69.0,H-MC,3.20,121.0,0.16,2.00,23.52,AR,NTT,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.23
1,25,44.63
2,50,76.57


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.79
MC    31.02
LC    23.19
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.75
X40      23.00
X40N     15.28
AR        8.97
XR        8.88
XY25      8.77
OX40N     7.47
SR        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.33
H-SC    23.00
X-LC    19.85
M-SC    11.59
X-SC    10.42
H-MC     4.92
M-MC     1.39
H-LC     1.13
L-LC     1.12
M-LC     1.09
L-SC     0.78
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.01,-13.45,51.58
FINANCE,13.52,-11.15,54.89
IT,11.63,-32.12,98.13
MISC,7.08,-31.80,85.71
ELECTRICAL,6.16,-9.60,49.58
PAINTS,5.06,-31.04,50.50
INSURANCE,4.83,-0.98,35.54
PHARMA,4.15,-1.94,34.03
AUTO,3.19,-18.19,59.07


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3253161.0,21
AR,1323842.0,10
X40,1297678.0,15
XR,1255505.0,12
X40N,995546.0,10
OX40N,751985.0,10
XY25,400395.0,6
SR,297710.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3596414.0,24
X-MC,1641274.0,16
M-SC,1445799.0,15
X-LC,1130549.0,12
X-SC,842798.0,9
H-MC,394632.0,3
L-SC,181765.0,2
M-LC,139840.0,1
H-LC,76018.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1227752.0      6
           AR         942870.0      5
M-SC       XY24       816757.0      6
H-SC       XR         792958.0      7
X-LC       X40        587108.0      6
X-MC       X40        540567.0      7
           XY24       487847.0      3
           X40N       405473.0      4
X-SC       X40N       365836.0      4
H-SC       OX40N      335124.0      4
M-SC       OX40N      328217.0      5
X-SC       XY24       306959.0      3
H-SC       SR         297710.0      2
X-LC       X40N       224237.0      2
H-MC       XY24       210923.0      1
X-MC       XY25       207387.0      2
X-LC       XY24       202923.0      2
H-MC       AR         183709.0      2
M-SC       XR         179580.0      2
X-SC       X40        170003.0      2
M-LC       XR         139840.0      1
M-SC       AR         121245.0      2
X-LC       XY25       116281.0      2
L-SC       XR          93121.0      1
           OX40N       88644.0      1
H-LC       AR          76018.0      1
M-MC       XY25        52438.0      1
L-MC       XR          50006.0      1
L-LC       XY25        24289.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 23.0 seconds
